# Step3 Analyze Street and County Data

In [2]:
# Import the data
from tensorflow.keras import layers, models, Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
import torch


df = pd.read_csv('./Resources/result_files/step1_build_base_data.csv')   # Read in data from step1
df.head()

,Unnamed: 0,County,Severity,Start_Time,Street,Weather_Condition,Start_Time_int,ID
0,0,1,3,2022-09-08 13:03:18,I-25 N,Partly Cloudy,20220908,57335
1,1,59,1,2022-09-08 08:44:08,W 44th Ave,Fair,20220908,57362
2,2,1,2,2022-09-08 06:57:38,E 112th Ave,Fair,20220908,57373
3,3,5,1,2022-09-08 08:45:51,S Tower Rd,Fair,20220908,57374
4,4,31,3,2022-09-08 08:42:23,I-25,Fair,20220908,57375


In [2]:
df.info()    # display column, null, and Dtype info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90885 entries, 0 to 90884
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         90885 non-null  int64 
 1   County             90885 non-null  int64 
 2   Severity           90885 non-null  int64 
 3   Start_Time         90885 non-null  object
 4   Street             90727 non-null  object
 5   Weather_Condition  89475 non-null  object
 6   Start_Time_int     90885 non-null  int64 
 7   ID                 90885 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 5.5+ MB


In [3]:
df['Severity'].value_counts()              # find and display out severity counts

Severity
2    56105
3    26306
4     7504
1      970
Name: count, dtype: int64

In [4]:
df.isna().sum()                            # find and display null values

Unnamed: 0              0
County                  0
Severity                0
Start_Time              0
Street                158
Weather_Condition    1410
Start_Time_int          0
ID                      0
dtype: int64

In [5]:
df = df.dropna(subset=['Street'])           # drop null values in Street column
df.isna().sum()


Unnamed: 0              0
County                  0
Severity                0
Start_Time              0
Street                  0
Weather_Condition    1408
Start_Time_int          0
ID                      0
dtype: int64

In [6]:
df['Street'].value_counts()                 # find and display street counts

Street
I-25 N                  8585
I-25 S                  7708
I-70 E                  5211
I-70 W                  4979
US-87 N                 2394
                        ... 
Duckwood Rd                1
Evergreen Rd               1
S Fiddlers Green Cir       1
Rusty Hinge Dr             1
 E 112th Ave               1
Name: count, Length: 4744, dtype: int64

In [7]:
df['Street'].nunique()                      # find and display unique street count

4744

In [8]:
# What top 30 streets have the most accidents?
df[["Street"]].value_counts().head(30)

Street               
I-25 N                   8585
I-25 S                   7708
I-70 E                   5211
I-70 W                   4979
US-87 N                  2394
I-25                     2187
US-87 S                  1897
US-6 W                   1383
I-70                     1064
I-225 N                  1017
I-225 S                   982
I-76 E                    866
US-6 E                    862
I-76 W                    699
CO-470 W                  692
CO-470 E                  634
US-36 E                   601
S Parker Rd               511
I-270 W                   504
 US Highway 6             450
US-85 S                   439
US-36                     428
 US Highway 285           415
US-36 W                   403
US-6                      375
Denver Boulder Tpke E     366
 US Highway 50            362
Federal Blvd              352
CO-470                    341
Denver Boulder Tpke W     333
Name: count, dtype: int64

In [9]:
# Store the street names of the top 30 with accidents in a list.
top30 = []
streets = df["Street"].value_counts().head(30)
top30 = streets.index.values[0:30].tolist()
print(top30)


['I-25 N', 'I-25 S', 'I-70 E', 'I-70 W', 'US-87 N', 'I-25', 'US-87 S', 'US-6 W', 'I-70', 'I-225 N', 'I-225 S', 'I-76 E', 'US-6 E', 'I-76 W', 'CO-470 W', 'CO-470 E', 'US-36 E', 'S Parker Rd', 'I-270 W', ' US Highway 6', 'US-85 S', 'US-36', ' US Highway 285', 'US-36 W', 'US-6', 'Denver Boulder Tpke E', ' US Highway 50', 'Federal Blvd', 'CO-470', 'Denver Boulder Tpke W']


In [10]:
# get top30 streets with accidents in CO and store in a dataframe called top30_df
top30_df = df.loc[df['Street'].isin(top30)]
# reset the index of top30_df
top30_df = top30_df.reset_index(drop=True)
top30_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47040 entries, 0 to 47039
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         47040 non-null  int64 
 1   County             47040 non-null  int64 
 2   Severity           47040 non-null  int64 
 3   Start_Time         47040 non-null  object
 4   Street             47040 non-null  object
 5   Weather_Condition  46399 non-null  object
 6   Start_Time_int     47040 non-null  int64 
 7   ID                 47040 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.9+ MB


In [11]:
  # extract relevant columns and display 'em

relevant_df = top30_df[['Street','County', 'Severity']]

[print('accident_count = ', relevant_df.count())]


accident_count =  Street      47040
County      47040
Severity    47040
dtype: int64


[None]

In [12]:
# reset the index of relevant_df
relevant_df = relevant_df.reset_index(drop=True)
relevant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47040 entries, 0 to 47039
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Street    47040 non-null  object
 1   County    47040 non-null  int64 
 2   Severity  47040 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [13]:
relevant_df['Severity'].value_counts()  # find and display severity counts (accidents by severity)

Severity
2    24816
3    19139
4     2942
1      143
Name: count, dtype: int64

In [14]:
relevant_df['County'].value_counts()  # find and display county counts (accidents by county)

County
31     11163
1       8343
59      4521
35      4240
41      3136
5       2938
69      1654
37      1573
123     1511
19      1376
45      1277
101     1217
117     1044
77       465
14       459
13       439
71       284
55       247
93       245
63       141
73       104
75        93
43        87
51        77
89        75
15        73
87        69
39        65
115       36
99        35
11        18
121       16
109       10
95         8
85         1
Name: count, dtype: int64

In [15]:
# Update CO_crashes_df with numeric county code

county_dict = {
"Adams":1,"Alamosa":3,"Arapahoe":5,"Archuleta":7,"Baca":9,"Bent":11,"Boulder":13,"Broomfield":14,"Chaffee":15,"Cheyenne":17,"Clear Creek":19,"Conejos":21,"Costilla":23,"Crowley":25,"Custer":27,
"Delta":29,"Denver":31,"Dolores":33,"Douglas":35,"Eagle":37,"El Paso":41,"Elbert":39,"Fremont":43,"Garfield":45,"Gilpin":47,"Grand":49,"Gunnison":51,"Hinsdale":53,"Huerfano":55,
"Jackson":57,"Jefferson":59,"Kiowa":61,"Kit Carson":63,"La Plata":67,"Lake":65,"Larimer":69,"Las Animas":71,"Lincoln":73,"Logan":75,"Mesa":77,"Mineral":79,
"Moffat":81,"Montezuma":83,"Montrose":85,"Morgan":87,"Otero":89,"Ouray":91,"Park":93,"Phillips":95,"Pitkin":97,"Prowers":99,"Pueblo":101,"Rio Blanco":103,"Rio Grande":105,"Routt":107,
"Saguache":109,"San Juan":111,"San Miguel":113,"Sedgwick":115,"Summit":117,"Teller":119,"Washington":121,"Weld":123,"Yuma":125
}

CO_crashes_df_updated =  relevant_df.replace({"County": county_dict})

CO_crashes_df_updated.head(5)


,Street,County,Severity
0,I-25 N,1,3
1,I-25,31,3
2,US-36,59,2
3,I-25 S,31,2
4,I-25 N,41,3


In [16]:
CO_crashes_df_updated['Severity'].value_counts()  # find and display severity counts (accidents by severity)

Severity
2    24816
3    19139
4     2942
1      143
Name: count, dtype: int64

In [17]:
# Preprocess Street Data - Encode top 30 streets by accidents

# Preprocess "Street" column (one-hot encoding)
street_encoder = OneHotEncoder(sparse_output=False)
street_encoded = street_encoder.fit_transform(CO_crashes_df_updated[['Street']])
street_columns = street_encoder.get_feature_names_out(['Street'])
df_street_encoded = pd.DataFrame(street_encoded, columns=street_columns)
df_street_encoded.head(5)

,Street_ US Highway 285,Street_ US Highway 50,Street_ US Highway 6,Street_CO-470,Street_CO-470 E,Street_CO-470 W,Street_Denver Boulder Tpke E,Street_Denver Boulder Tpke W,Street_Federal Blvd,Street_I-225 N,...,Street_S Parker Rd,Street_US-36,Street_US-36 E,Street_US-36 W,Street_US-6,Street_US-6 E,Street_US-6 W,Street_US-85 S,Street_US-87 N,Street_US-87 S
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_street_encoded.info()  # display column, null, and Dtype info for encoded street data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47040 entries, 0 to 47039
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Street_ US Highway 285        47040 non-null  float64
 1   Street_ US Highway 50         47040 non-null  float64
 2   Street_ US Highway 6          47040 non-null  float64
 3   Street_CO-470                 47040 non-null  float64
 4   Street_CO-470 E               47040 non-null  float64
 5   Street_CO-470 W               47040 non-null  float64
 6   Street_Denver Boulder Tpke E  47040 non-null  float64
 7   Street_Denver Boulder Tpke W  47040 non-null  float64
 8   Street_Federal Blvd           47040 non-null  float64
 9   Street_I-225 N                47040 non-null  float64
 10  Street_I-225 S                47040 non-null  float64
 11  Street_I-25                   47040 non-null  float64
 12  Street_I-25 N                 47040 non-null  float64
 13  S

In [19]:
df_street_encoded.head(5)

,Street_ US Highway 285,Street_ US Highway 50,Street_ US Highway 6,Street_CO-470,Street_CO-470 E,Street_CO-470 W,Street_Denver Boulder Tpke E,Street_Denver Boulder Tpke W,Street_Federal Blvd,Street_I-225 N,...,Street_S Parker Rd,Street_US-36,Street_US-36 E,Street_US-36 W,Street_US-6,Street_US-6 E,Street_US-6 W,Street_US-85 S,Street_US-87 N,Street_US-87 S
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_street_encoded = pd.concat([CO_crashes_df_updated, df_street_encoded], axis=1) # Merge the databases

In [21]:
df_street_encoded.info()  # display column, null, and Dtype info for merged data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47040 entries, 0 to 47039
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Street                        47040 non-null  object 
 1   County                        47040 non-null  int64  
 2   Severity                      47040 non-null  int64  
 3   Street_ US Highway 285        47040 non-null  float64
 4   Street_ US Highway 50         47040 non-null  float64
 5   Street_ US Highway 6          47040 non-null  float64
 6   Street_CO-470                 47040 non-null  float64
 7   Street_CO-470 E               47040 non-null  float64
 8   Street_CO-470 W               47040 non-null  float64
 9   Street_Denver Boulder Tpke E  47040 non-null  float64
 10  Street_Denver Boulder Tpke W  47040 non-null  float64
 11  Street_Federal Blvd           47040 non-null  float64
 12  Street_I-225 N                47040 non-null  float64
 13  S

In [22]:
# drop Street from df_street_encoded
df_street_encoded = df_street_encoded.drop(columns=['Street'])  # drop street from the result for pvalue test

In [23]:
pvalues = df_street_encoded.corr()['Severity'].sort_values(ascending=False)   # obtain p-values for the top 30 streets
print(pvalues)

Severity                        1.000000
Street_ US Highway 6            0.095258
Street_US-87 N                  0.063802
Street_I-25 N                   0.059105
Street_US-87 S                  0.054434
Street_ US Highway 50           0.039263
Street_I-25 S                   0.031770
Street_ US Highway 285          0.023408
Street_US-85 S                  0.020724
Street_I-270 W                  0.019563
Street_US-6 W                   0.012349
Street_I-70 E                   0.008659
Street_I-76 E                   0.008196
Street_I-70 W                   0.007905
Street_I-225 N                  0.002867
Street_I-225 S                  0.001835
Street_I-76 W                   0.001507
Street_US-36 W                  0.000317
County                         -0.000599
Street_US-6 E                  -0.015143
Street_US-36 E                 -0.017150
Street_Denver Boulder Tpke W   -0.039935
Street_Federal Blvd            -0.041269
Street_US-6                    -0.043159
Street_Denver Bo

In [33]:
pV_Streets = ['Street_ US Highway 50']  # set street to analyze with pvalue <.05
# copy only those columns in list P-Value_Streets and drop the rest of the columns from df_street_encoded
df_street_encoded_PV = df_street_encoded[pV_Streets]
df_street_encoded_PV.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47040 entries, 0 to 47039
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Street_ US Highway 50  47040 non-null  float64
dtypes: float64(1)
memory usage: 367.6 KB


In [26]:
df_street_encoded_PV.value_counts()  # find and display encoded counts (0 = row without an accident; 1 = row with accident)

Street_ US Highway 50
0.0                      46678
1.0                        362
Name: count, dtype: int64

In [27]:
# define X
X = CO_crashes_df_updated.drop(columns=['County', 'Street'])   # Set X to Serverity

# Preprocess "county" column (label encoding for binary; one-hot encoding for multiple categories)
county_encoder = LabelEncoder()
df_street_encoded['County'] = county_encoder.fit_transform(df_street_encoded['County'])

y_county = df_street_encoded['County']

y_street = df_street_encoded[pV_Streets]

# Split data into training and testing sets

X_train, X_test, y_county_train, y_county_test, y_street_train, y_street_test = train_test_split(X, y_county, y_street)

In [28]:
# Create the shared layers of the model

# Input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Shared hidden layers
shared_layer1 = layers.Dense(128, activation='relu')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu')(shared_layer1)

In [29]:
# Branch for street prediction
street_output = layers.Dense(1, activation='sigmoid', name='street_output')(shared_layer2)

# Branch for county prediction
county_output = layers.Dense(1, activation='sigmoid', name='county_output')(shared_layer2)

In [30]:
# Create the model
model = Model(inputs=input_layer, outputs=[street_output, county_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'county_output': 'binary_crossentropy', 'street_output': 'binary_crossentropy'},
              metrics={'county_output': 'accuracy', 'street_output': 'accuracy'})

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 dense (Dense)               (None, 128)                  256       ['input_features[0][0]']      
                                                                                                  
 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 street_output (Dense)       (None, 1)                    65        ['dense_1[0][0]']         

In [31]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 1) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
input_features [(None, 1)] float32
dense (None, 1) float32
dense_1 (None, 128) float32
street_output (None, 64) float32
county_output (None, 64) float32


[None, None, None, None, None]

In [32]:
# Fit the model
from timeit import default_timer as timer
start_time = timer()
model.fit(
    X,
    {'county_output': y_county, 'street_output': y_street},
    epochs=50,
    batch_size=32,
    validation_split=0.2
)
elapsed_time = timer() - start_time

print('Fit Model Time = ',elapsed_time)

Epoch 1/50
1176/1176 [==============================] - 5s 3ms/step - loss: -117550.6094 - street_output_loss: 0.2469 - county_output_loss: -117550.8594 - street_output_accuracy: 0.9917 - county_output_accuracy: 0.0626 - val_loss: -482966.4688 - val_street_output_loss: 1.4233 - val_county_output_loss: -482967.9375 - val_street_output_accuracy: 0.9881 - val_county_output_accuracy: 0.0619
Epoch 2/50
1176/1176 [==============================] - 4s 4ms/step - loss: -1493042.1250 - street_output_loss: 2.2127 - county_output_loss: -1493045.0000 - street_output_accuracy: 0.9874 - county_output_accuracy: 0.0626 - val_loss: -3158081.7500 - val_street_output_loss: 0.4512 - val_county_output_loss: -3158082.5000 - val_street_output_accuracy: 0.9881 - val_county_output_accuracy: 0.0619
Epoch 3/50
1176/1176 [==============================] - 3s 3ms/step - loss: -5413382.0000 - street_output_loss: 5.4577 - county_output_loss: -5413388.0000 - street_output_accuracy: 0.9849 - county_output_accuracy: 0.

In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'county_output': y_county_test, 'street_output': y_street_test})
test_results

368/368 [==============================] - 1s 2ms/step - loss: -11081693184.0000 - street_output_loss: 409.6661 - county_output_loss: -11081693184.0000 - street_output_accuracy: 0.9916 - county_output_accuracy: 0.0611


[-11081693184.0,
 409.66607666015625,
 -11081693184.0,
 0.9915816187858582,
 0.06113945692777634]

In [ ]:
# Print the accuracy
print(f"Street Accuracy: {test_results[3]}")
print(f"County Accuracy: {test_results[4]}")

Street Accuracy: 0.9915816187858582
County Accuracy: 0.06113945692777634


In [36]:
# Run statistics for street prediction with a street p-value less than .05

# Street-  US Highway 50   Accuracy- 0.9915816187858582   p-Value - 0.039263   Value Counts 0.0=46678   1.0= 362
# Street-  I-25 S          Accuracy- 0.8414965867996216   p-Value - 0.031770   Value Counts 0.0=39332   1.0=7708
# Street-  US Highway 285  Accuracy- 0.9923469424247742   p-Value - 0.023408   Value Counts 0.0=46625   1.0= 415
# Street-  US-85 S         Accuracy- 0.989625871181488    p-Value - 0.020724   Value Counts 0.0=46601   1.0= 439
# Street-  I-270 W         Accuracy- 0.9897959232330322   p-Value - 0.019563   Value Counts 0.0=46536   1.0= 504
# Street-  US-6 W          Accuracy- 0.9717687368392944   p-Value - 0.012349   Value Counts 0.0=45657   1.0=1383
# Street-  I-70 E          Accuracy- 0.8852040767669678   p-Value - 0.008659   Value Counts 0.0=41829   1.0=5211
# Street-  I-70 W          Accuracy- 0.8955782055854797   p-Value - 0.007905   Value Counts 0.0=42061   1.0=4979
# Street-  I-225 N         Accuracy- 0.9786564707756042   p-Value - 0.002867   Value Counts 0.0=46023   1.0=1017
# Street-  I-225 S         Accuracy- 0.9795918464660645   p-Value - 0.001835   Value Counts 0.0=46058   1.0= 982
# Street-  I-76 W          Accuracy- 0.9854592084884644   p-Value - 0.001507   Value Counts 0.0=46341   1.0= 669
# Street-  US-36 W         Accuracy- 0.9915816187858582   p-Value - 0.000317   Value Counts 0.0=46637   1.0= 403
